In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from module.components.discrete_gaussian1D import DiscreteGaussian1D
from module.components.lawrence_dist import LawrenceDist

## Distribution Test Site
It is tried to find a discrete two-dimensional distribution given its mean and variance. Due to theoretical reasoning, and maximizing entropy under constrains, the functional form of the distribution $p(n,m)$ for $n,m \in \mathbb{Z}$ is:

$$
p(n, m) = \frac{1}{Z}e^{-(\frac{n - \mu_1}{\alpha_1})^2 -(\frac{m - \mu_2}{\alpha_2})^2 + \lambda n m}
$$

In [3]:
phase_space = np.stack(np.meshgrid(np.arange(-20, 20), np.arange(-20, 20)), axis = -1)

In [4]:
def calc_prob(mean_n, mean_m, var_n, var_m, cov):
    thresh = 0.3
    if var_n < thresh:
        var_n = thresh
    if var_m < thresh:
        var_m = thresh

    if cov**2 >= var_n * var_m * 0.95:
       cov = np.sign(cov) * np.sqrt(var_n * var_m) * 0.95

    cov_matrix = np.array([[var_n, cov],[cov, var_m]])
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    x = phase_space - np.array([mean_n, mean_m])
    q = np.squeeze(inv_cov_matrix @ np.expand_dims(x, axis = -1))
    exp = -0.5 * np.sum(x * q, axis = -1)
    probs = np.exp(exp)
    Z = np.sum(probs)

    if Z < 1e-3:
        print("emergency")
        return calc_prob(mean_n, mean_m, var_n, var_n, cov * 0.95)

    return probs / Z

In [5]:
#target values
t_mean_n = -5.654
t_mean_m = 2.42
t_var_n = 0.377
t_var_m = 0.356
t_cov = 0.72 * np.sqrt(t_var_n * t_var_m)

#initial parameters
p_mean_n = t_mean_n
p_mean_m = t_mean_m
p_var_n = t_var_n
p_var_m = t_var_m
p_cov = t_cov

dt = 0.5
probs = None
for i in range(15):
    #current values
    probs = calc_prob(p_mean_n, p_mean_m, p_var_n, p_var_m, p_cov)
    mean_n = np.sum(phase_space[:,:,0] * probs)
    mean_m = np.sum(phase_space[:,:,1] * probs)
    var_n = np.sum(phase_space[:,:,0] ** 2 * probs) - mean_n **2
    var_m = np.sum(phase_space[:,:,1] **2 * probs) - mean_m **2
    cov =  np.sum(phase_space[:,:,0] * phase_space[:,:,1] * probs) - mean_n * mean_m

    #deltas
    d_mean_n = t_mean_n - mean_n
    d_mean_m = t_mean_m - mean_m
    d_var_n = t_var_n - var_n
    d_var_m = t_var_m - var_m
    d_cov = t_cov - cov

    #apply
    p_mean_n += dt * d_mean_n
    p_mean_m += dt * d_mean_m
    p_var_n += dt * d_var_n
    p_var_m += dt * d_var_m
    p_cov += dt * d_cov

    #restrict
    if p_var_n < 0:
        p_var_n = 0

    if p_var_m < 0:
        p_var_m = 0


In [6]:
probs = calc_prob(p_mean_n, p_mean_m, p_var_n, p_var_m, p_cov)
mean_n = np.sum(phase_space[:,:,0] * probs)
mean_m = np.sum(phase_space[:,:,1] * probs)
var_n = np.sum(phase_space[:,:,0] ** 2 * probs) - mean_n **2
var_m = np.sum(phase_space[:,:,1] **2 * probs) - mean_m **2
cov =  np.sum(phase_space[:,:,0] * phase_space[:,:,1] * probs) - mean_n * mean_m

print("mean n:", mean_n)
#print("param mean n:", p_mean_n)
print("target mean n:", t_mean_n,"\n")

print("mean m:", mean_m)
#print("param mean m:", p_mean_m)
print("target mean m:", t_mean_m,"\n")

print("var n:", var_n)
#print("param var n:", p_var_n)
print("target var n:", t_var_n,"\n")

print("var m:", var_m)
#print("param var m:", p_var_m)
print("target var m:", t_var_m,"\n")

print("cov:", cov)
#print("param cov:", p_cov)
print("target cov:", t_cov,"\n")

mean n: -5.65399981252634
target mean n: -5.654 

mean m: 2.4199995276627893
target mean m: 2.42 

var n: 0.37700043739123146
target var n: 0.377 

var m: 0.35600199897293283
target var m: 0.356 

cov: 0.26377280928103986
target cov: 0.2637716830897509 

